# Extract various interesting CME times from 
# COR2 (Stereo) Catalog (Angelos Vourlidas etal, 2017+) of CME

Then compare with the LASCO C2 catalog times to get catalog of images we want to use; they wont match 100% as COR2 is for Stereo A/B which have differnt orbit/viewing of Sun (generally) than SOHO. We'll need to take into account the "solar longitude" to make sure that the CME is one which SOHO could have observed (from L2 Halo orbit on Sun-Earth line).

In [1]:
import pandas as pd

Need_To_Redownload = False

# path to the CSV files (by year) which contain COR2 catalog
COR2_data_path = '../COR2_catalog_20170927'

# name of the LASCO catalog
LASCO_Catalog_Filename = '../lasco_img_catalog_lvl_1.txt'

CME_Catalog_Filename = '../cme_event_catalog.txt'

In [3]:


# find all files in COR2 directory
# and load into memory

from os import listdir
from os.path import isfile, join

files = [f for f in listdir(COR2_data_path) if isfile(join(data_path, f))]


df = pd.read_csv(join(data_path, files[0]))
foo = df[df.Comments.str.contains('SB') == True]
foo


FileNotFoundError: [Errno 2] No such file or directory: '../COR2_catalog_20170927'

In [ ]:
# pull the cme catalog to local disk
if Need_To_Redownload:
    import requests
    
    # download LASCO catalog now
    #lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/img_hdr.txt'   # level 0.5 data
    lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/level_1/img_hdr.txt'
    response = requests.get(lasco_hdr_url, allow_redirects=True)
    
    # cache locally
    open(LASCO_Catalog_Filename, 'wb').write(response.content)
    

# load data from LASCO img catalog
lasco_catalog = pd.read_fwf(LASCO_Catalog_Filename, index=False,
                            names=['filename', 'date-obs', 'time-obs','telescope', 'expTime',
                                   'NumCols', 'NumRows', 'StartCol', 'StartRow', 'Filter', 
                                   'Polarizer','Observ.Seq', 'LEB Program', 'junk'])

lasco_catalog
c2_catalog = lasco_catalog[lasco_catalog['telescope'] == 'C2']
#c2_catalog

In [ ]:
slice1 = c2_catalog[c2_catalog['date-obs']=='2014/01/09']
slice1 = slice1[slice1['time-obs'].str.contains(':37:') == True]
slice1

In [ ]:
cdaw_data = pd.read_csv(CME_Catalog_Filename)
cdaw_data = cme_data.rename(columns={'Unnamed: 0':'CDAW_Id'})
cdaw_data

In [ ]:
test = cdaw_data[cdaw_data['Date'] == '2014/01/09']
#test = test[test['Time'].str.contains(':37:') == True]
test